# Preprocessing and Data Splitting/Sampling
This notebook includes all the steps necessary to preprocess, sample and split the COVID Teacher Latin America data into training and test sets.

# Load in Dataset and Explore Schema

In [1]:
from pyspark.sql import *
DATA_FILEPATH = 'data/clean_data.csv'

spark = SparkSession \
    .builder \
    .appName("Preprocessing") \
    .getOrCreate()

df = spark.read.csv(DATA_FILEPATH,  inferSchema=True, header = True)
from pyspark.sql.functions import when

In [2]:
df.show(2)

+---+----+---+------------+---------------+------------------+------------------+--------------+-------------+-----------------+---------------+-----------------------+----------------------+-------------------+----------+-------+-------------+---------+-----------+---------------------+--------------------+-----------------------+-----------------+------------+-------------+------------+--------------+--------------+---------------+---------------+--------------+------------------+-----------------+------------------+-----------------+-----------------+---+---------------------+-------------+---------+-----------------+-------+---------------+----------------------------------------------------------------------------------------+---+-----------------------+---------+----------+-----------+-----------+
|_c0| Age|SES|RuralVsUrban|EducationDegree|SecondaryVsPrimary|TeachPublicVsOther|YearsAsTeacher|EmployedVsNot|PastCOVIDpositive|COVIDvaccinated|PrePandemicChronIllness|PrePandemicMentIl

In [3]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Age: double (nullable = true)
 |-- SES: double (nullable = true)
 |-- RuralVsUrban: double (nullable = true)
 |-- EducationDegree: double (nullable = true)
 |-- SecondaryVsPrimary: double (nullable = true)
 |-- TeachPublicVsOther: double (nullable = true)
 |-- YearsAsTeacher: double (nullable = true)
 |-- EmployedVsNot: double (nullable = true)
 |-- PastCOVIDpositive: double (nullable = true)
 |-- COVIDvaccinated: double (nullable = true)
 |-- PrePandemicChronIllness: double (nullable = true)
 |-- PrePandemicMentIllness: double (nullable = true)
 |-- PrePandemicNeuroDis: double (nullable = true)
 |-- Depression: double (nullable = true)
 |-- Anxiety: double (nullable = true)
 |-- OverallHealth: double (nullable = true)
 |-- COVIDfear: double (nullable = true)
 |-- RelatImprov: double (nullable = true)
 |-- WorkloadNowVsPreCOVID: double (nullable = true)
 |-- ResourceSatisfaction: double (nullable = true)
 |-- SufficientCOVIDmeasures: double

# Drop Unnecessary Variables and Rename Non-University Studies
These variables are collinear with other variables, so we will drop them. 

In [4]:
df = df.drop("Women/Trans-Nonbinary")
df = df.drop("Not Partnered")
df = df.drop('StudentProblems')
df = df.drop('_c0')
df = df.withColumnRenamed('Non-University Studies (e.g., Vocational Training, Early Childhood Education Technician)', 'Non_university_studies')

# Binning Outcome Variables

In [5]:
df = df.withColumn('Depression', when(df['Depression'] >= 3, 1).otherwise(0))
df = df.withColumn('Anxiety', when(df['Anxiety'] >= 3, 1).otherwise(0))

In [6]:
Anxiety = df.select(df['Anxiety'])
Anxiety.show(10)

+-------+
|Anxiety|
+-------+
|      0|
|      1|
|      1|
|      1|
|      0|
|      0|
|      0|
|      1|
|      1|
|      0|
+-------+
only showing top 10 rows



In [7]:
Depression = df.select(df['Depression'])
Depression.show(10)

+----------+
|Depression|
+----------+
|         0|
|         1|
|         0|
|         1|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
+----------+
only showing top 10 rows



# Sampling and Splitting

In [8]:
df.groupBy('Anxiety').count().show()

+-------+-----+
|Anxiety|count|
+-------+-----+
|      1|  737|
|      0| 1267|
+-------+-----+



In [9]:
df.groupBy('Depression').count().show()

+----------+-----+
|Depression|count|
+----------+-----+
|         1|  279|
|         0| 1725|
+----------+-----+



In [10]:
train, test = df.randomSplit([0.85, 0.15], 314)
def dynamic_upsample_spark(df, target, positive_label, negative_label):
    positive_count = df.filter(df[f'{target}']==positive_label).count()
    negative_count = df.filter(df[f'{target}']==negative_label).count()
    total = df.count()
    positive_fraction = positive_count/total
    negative_fraction = negative_count/total
    pos_to_neg = positive_count/negative_count #when neg>pos, so upsample pos (or downsample neg)
    neg_to_pos = negative_count/positive_count #when neg<pos, so downsample pos
    if positive_fraction < negative_fraction: #upsample pos
        df2 = df.filter(df[f"{target}"]==positive_label).sample(fraction = neg_to_pos, withReplacement=True, seed=1)
        df_final = df.filter(df[f'{target}']==negative_label).union(df2) 
    if positive_fraction > negative_fraction: #upsample neg
        df2 = df.filter(df[f"{target}"]==negative_label).sample(fraction = pos_to_neg, withReplacement=True, seed=1)
        df_final = df.filter(df[f'{target}']==positive_label).union(df2) 
    return df_final

In [11]:
#Resample Anxiety and prove it worked
Anxiety_df = dynamic_upsample_spark(train, 'Anxiety', 1, 0)
Anxiety_df.groupBy('Anxiety').count().show()

+-------+-----+
|Anxiety|count|
+-------+-----+
|      0| 1077|
|      1| 1051|
+-------+-----+



In [12]:
#Resample Depression and prove it worked
Depression_df = dynamic_upsample_spark(train, 'Depression', 1, 0)
Depression_df.groupBy('Depression').count().show()

+----------+-----+
|Depression|count|
+----------+-----+
|         0| 1452|
|         1| 1440|
+----------+-----+



# Export Dataframes to CSV Files

In [13]:
Anxiety_df.toPandas().to_csv('data/anxiety.csv')
Depression_df.toPandas().to_csv('data/depression.csv')
test.toPandas().to_csv('data/test.csv')